In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import cv2

from tensorflow.keras.layers import Input, Conv2D 
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense 
from tensorflow.keras import Model
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import seaborn as sns

In [2]:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
tf.__version__


'2.7.0'

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
path = "dataset_sidi_512.csv"
df512 = pd.read_csv(path, sep='\t')
df512

,Unnamed: 0,Unnamed: 0.1,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,382583,382583,798338609870872577-2,0.774375,0.174701,0.050924,what does it look like i do for a living? (cra...,NEG,0,data/79833/798338609870872577-2.jpg,1280,722
1,443004,443004,802556641057054721-1,0.540009,0.304103,0.155888,No cheat just skill. #ClikerHeroes https://t.c...,NEG,0,data/80255/802556641057054721-1.jpg,707,1366
2,348885,348885,796032212407721984-1,0.513661,0.322456,0.163883,@KEILOin_DaTrunk I deleted https://t.co/qIhBkn...,NEG,0,data/79603/796032212407721984-1.jpg,1334,750
3,377487,377487,798011160532381696-1,0.712648,0.254173,0.033180,"""RT NYTFashion: How Nasty Gal went from an eBa...",NEG,0,data/79801/798011160532381696-1.jpg,561,1000
4,423703,423703,801181521679790080-1,0.713677,0.186854,0.099468,I hate this nigga lmaooo https://t.co/gPcyJfJESN,NEG,0,data/80118/801181521679790080-1.jpg,1334,750
...,...,...,...,...,...,...,...,...,...,...,...,...
11995,108284,108284,769551952275509248-1,0.010517,0.886504,0.102978,RT @Alwas97: find someone who loves you like b...,NEU,1,data/76955/769551952275509248-1.jpg,1218,1078
11996,93186,93186,769486781158858753-1,0.065240,0.897818,0.036942,RT @MelbWritersFest: Helen Garner talking to @...,NEU,1,data/76948/769486781158858753-1.jpg,1491,2048
11997,298145,298145,783915769943973890-1,0.011468,0.925276,0.063257,Watch lesbian movie: https://t.co/jzJrDcnAwaSh...,NEU,1,data/78391/783915769943973890-1.jpg,1024,683
11998,260672,260672,783138720610476033-1,0.003072,0.957911,0.039016,Click here to watch the movie: https://t.co/A...,NEU,1,data/78313/783138720610476033-1.jpg,1024,682


In [6]:
filenames = ['./Dataset512/' + fname for fname in df512['image_name'].tolist()]
labels = df512['sent_image'].tolist()
train_filenames, val_filenames, train_labels, val_labels = train_test_split(filenames,labels, train_size=0.9,random_state=42)

In [7]:
from keras.utils.np_utils import to_categorical   

train_labels = to_categorical(train_labels, num_classes=3)
test_labels = to_categorical(val_labels, num_classes=3)

In [8]:
width = 128
height = 128

X_train = np.zeros((int(train_labels.shape[0]), width, height, 3))
x_test = np.zeros((int(test_labels.shape[0]), width, height, 3))
aux =0

dim = (width, height)

for i in train_filenames:
    X_train[aux] = np.asarray(cv2.resize(cv2.imread(i+'.jpg'), dim, interpolation = cv2.INTER_AREA) )
    aux = aux + 1
aux =0
for i in val_filenames:
    x_test[aux] = np.asarray(cv2.resize(cv2.imread(i+'.jpg'), dim, interpolation = cv2.INTER_AREA) )
    aux = aux + 1

In [9]:
X_train = X_train.reshape((X_train.shape[0],) +(width, height,3)).astype('float32') / 255.
x_test = x_test.reshape((x_test.shape[0],) +(width, height,3)).astype('float32') / 255.

In [10]:
X_train.shape,x_test.shape

((10800, 128, 128, 3), (1200, 128, 128, 3))

## VGG16


In [11]:
tf.keras.backend.clear_session()
import gc
gc.collect()

4

In [12]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
#base_model.trainable = False ## Not trainable weights

Lock nas camadas

In [13]:
# Freeze four convolution blocks
for layer in base_model.layers[:12]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [14]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [15]:
## Preprocessing input
#Mudei a variavél so por que era o que estava no codigo base
train_ds = preprocess_input(X_train) 
test_ds = preprocess_input(x_test)

In [16]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
prediction_layer = layers.Dense(3, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    prediction_layer
])

### Treino na mão

In [17]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
epocas = 1
batch_size = 16
for i in range(epocas):
    for j in range(0,train_ds.shape[0]-batch_size,batch_size):
        model.train_on_batch(train_ds[j:j+batch_size],train_labels[j:j+batch_size])

### Treino com fit

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=1,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=4, callbacks=[es])

Epoch 1/50
2160/2160 [==============================] - 40s 18ms/step - loss: 1.0989 - accuracy: 0.3255 - val_loss: 1.0986 - val_accuracy: 0.3352
Epoch 2/50
2160/2160 [==============================] - 38s 18ms/step - loss: 1.0989 - accuracy: 0.3339 - val_loss: 1.0988 - val_accuracy: 0.3282


### Resultados

In [18]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
y_pred =(y_pred>0.5)
#list(y_pred)
print(accuracy_score(test_labels, y_pred));
print(classification_report(test_labels, y_pred));

0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       408
           1       0.00      0.00      0.00       386
           2       0.00      0.00      0.00       406

   micro avg       0.00      0.00      0.00      1200
   macro avg       0.00      0.00      0.00      1200
weighted avg       0.00      0.00      0.00      1200
 samples avg       0.00      0.00      0.00      1200



C:\Users\bhla\anaconda3\envs\DP\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bhla\anaconda3\envs\DP\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bhla\anaconda3\envs\DP\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CNN básica


In [306]:
tf.keras.backend.clear_session()
import gc
gc.collect()

100

In [300]:
camadas = 32
model = models.Sequential()
model.add(layers.Conv2D(camadas, (3, 3), activation='relu', input_shape=(width, height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(camadas*2, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(camadas*4, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(camadas*8, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(3,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

### Treino na mão

In [312]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
epocas = 5
batch_size = 16
for i in range(epocas):
    for j in range(0,train_ds.shape[0]-batch_size,batch_size):
        model.train_on_batch(train_ds[j:j+batch_size],train_labels[j:j+batch_size])

### Treino com o fit

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_loss', mode='max', patience=1,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=16, callbacks=[es])

### Resultados

In [314]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
y_pred =(y_pred>0.5)
#list(y_pred)
print(accuracy_score(test_labels, y_pred));
print(classification_report(test_labels, y_pred));